In [5]:
import os
os.environ['SIRIUS_URL_CONSTS'] = 'http://127.0.0.1'

In [6]:
%gui qt5
%matplotlib qt5

from functools import partial as _partial
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
import matplotlib.gridspec as mpl_gs
import matplotlib.cm as cmap
import matplotlib.colors as mpl_colors
from matplotlib import rcParams
rcParams.update(
    {'font.size': 16,'lines.linewidth': 2, 'axes.grid': True})

import pyaccel
import mathphys
from pymodels import si
from apsuite.dynap import DynapXY

In [7]:
model = si.create_accelerator()
model.cavity_on = True
model.radiation_on = False
dynap = DynapXY(model)

In [8]:
dynap.params.turn_by_turn = True
dynap.params.x_nrpts = 20
dynap.params.y_nrpts = 5
dynap.params.x_min = -0.010
dynap.params.x_max = -0.006
dynap.params.y_min = 0
dynap.params.y_max = 0.001
print(dynap)

nrturns      : 512
turn_by_turn : True
x_nrpts      : 20
y_nrpts      : 5
x_min [m]    : -0.01
x_max [m]    : -0.006
y_min [m]    : 0
y_max [m]    : 0.001
de_offset    : 0



In [9]:
# dynap.do_tracking()

In [10]:
# dynap.process_data()

In [5]:
# # dynap.save_data('test.pickle')
# dynap.load_and_apply('test.pickle')
# dynap.process_data()

In [9]:
dynap.load_and_apply('/home/fernando/nominal.pickle')
dynap.process_data()

# Diffusion Plot

In [10]:
f = dynap.make_figure_diffusion(contour=True, orders=[1, 2, 3, ], symmetry=1, intnux=49, intnuy=14)
# f = dynap.make_figure_diffusion(contour=False)

# X and Y map to tune plot

In [11]:
f = dynap.make_figure_xandy_map_in_tune_plot(orders=[1, 2, 3, ], symmetry=1, intnux=49, intnuy=14)